 # ORB Detector Parameter Tuning 
## Setup the required functions and input images

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb  1 20:06:06 2019

@author: vik748
"""
import numpy as np
import cv2
import sys
from matplotlib import pyplot as plt
from ssc import *
%matplotlib inline
import time


def draw_keypoints(vis_orig, keypoints, color = (0, 255, 255), thick = 3):
    vis = vis_orig.copy()
    for kp in keypoints:
        x, y = kp.pt
        cv2.circle(vis, (int(x), int(y)), int(vis.shape[1]/200), color, thickness=thick)
    return vis

def draw_markers(vis_orig, keypoints, color = (0, 0, 255)):
    vis = vis_orig.copy()
    for kp in keypoints:
        x, y = kp.pt
        cv2.drawMarker(vis, (int(x), int(y)), color,  markerSize=30, markerType = cv2.MARKER_CROSS, thickness=3)
    return vis

def draw_marker_pts(vis_orig, pts, color = (0, 0, 255)):
    vis = vis_orig.copy()
    for row in pts[:,0,:]:
        cv2.drawMarker(vis, (int(row[0]), int(row[1])), color,  markerSize=30, markerType = cv2.MARKER_CROSS, thickness=3)
    return vis

if sys.platform == 'darwin':
    path = '/Users/vik748/Google Drive/'
else:
    path = '/home/vik748/'
img = cv2.imread(path+'data/Lars2_081018/G0028388.JPG',1) # iscolor = CV_LOAD_IMAGE_GRAYSCALE
img2 = cv2.imread(path+'data/Lars2_081018/G0028393.JPG',1) # iscolor = CV_LOAD_IMAGE_GRAYSCALE



mask1 = cv2.imread(path+'data/Lars2_081018_clahe_masks2_out/G0028388_mask.png',cv2.IMREAD_GRAYSCALE)
mask2 = cv2.imread(path+'data/Lars2_081018_clahe_masks2_out/G0028393_mask.png',cv2.IMREAD_GRAYSCALE)
#img = cv2.imread(path+'kitti/00/image_0/000000.png',1) # iscolor = CV_LOAD_IMAGE_GRAYSCALE
#img2 = cv2.imread(path+'kitti/00/image_0/000001.png',1) # iscolor = CV_LOAD_IMAGE_GRAYSCALE

#img = np.delete(img,-1,1)
#img2 = np.delete(img2,-1,1)

mask_pts_1498 = np.array([[1030, 740], [1820, 840], [1785, 1085], [1675, 1400], [975, 1285]])
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

clahe = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(8,6))
gray = clahe.apply(gray)
gray2 = clahe.apply(gray2)



# create a mask image filled with 1s, the size of original image
mask = np.ones(img.shape[:2], dtype=np.uint8)
mask = cv2.fillConvexPoly(mask, mask_pts_1498, color=[0, 0, 0])


## Default Settings on High Res Image

In [ ]:
detector1 = cv2.ORB_create(nfeatures=1000, scaleFactor=4.0, nlevels=4, edgeThreshold=125, 
                           firstLevel=0, WTA_K=4, scoreType=cv2.ORB_HARRIS_SCORE,  
                           patchSize=125, fastThreshold=5)

# find the keypoints and descriptors with ORB
kp1 = detector1.detect(gray,mask1)
kp2 = detector1.detect(gray2,mask2)

kp1_pts = np.array([o.pt for o in kp1],dtype='float32')
kp1_pts = np.expand_dims(kp1_pts,1)


img_out = draw_keypoints(img,kp1, thick=3)
img_out2 = draw_keypoints(img2,kp2, thick=3)

fig, (ax1,ax2) = plt.subplots(2,1, dpi=200)
fig.suptitle('4000x3000 Image default orb settings')
ax1.axis("off")
ax2.axis("off")
#ax1.imshow(cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB))
#ax2.imshow(cv2.cvtColor(img_out2, cv2.COLOR_BGR2RGB))
ax1.imshow(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR))
ax2.imshow(cv2.cvtColor(gray2, cv2.COLOR_GRAY2BGR))

plt.show()

In [ ]:
print(kp1_pts.shape)

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (65,65),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
color = np.random.randint(0,255,(10000,3))
# Create a mask image for drawing purposes
mask = np.zeros_like(img)

start=time.time()
p1, st, err = cv2.calcOpticalFlowPyrLK(gray, gray2, kp1_pts, None, **lk_params)
print("elapsed: ",time.time()-start)
# Select good points
good_new = p1[st==1]
good_old = kp1_pts[st==1]
# draw the tracks
for i,(new,old) in enumerate(zip(good_new,good_old)):
    a,b = new.ravel()
    c,d = old.ravel()
    mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
    img = cv2.circle(img,(a,b),5,color[i].tolist(),-1)
img_out = cv2.add(img,mask)
fig, (ax1) = plt.subplots(1,1, dpi=200)
ax1.imshow(img_out)

In [ ]:
kp1_pts.shape

In [ ]:
import numpy as np
import cv2

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 200,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (30,30),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 0.03))
# Create some random colors
color = np.random.randint(0,255,(250,3))
# Take first frame and find corners in it
#ret, old_frame = cap.read()
old_frame = np.copy(img)
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
if 1:
    #ret,frame = cap.read()
    frame = np.copy(img2)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img_out = cv2.add(frame,mask)
    fig, ax= plt.subplots(dpi=200)
    plt.imshow(img_out)
    #k = cv2.waitKey(30) & 0xff
    #if k == 27:
    #    break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cv2.destroyAllWindows()


In [ ]:
p0.dtype


In [ ]:
np.sum(st)